## Data processing automation
1. Code from writing excel session + improvement
2. Migration to .py script
3. Text editor (Atom)
4. Running a script (Anaconda prompt)

#### Writing an excel file

In [1]:
import pandas as pd

#### Datasets

In [2]:
# sales report
report = pd.read_csv('../datasets/output_sales_report.csv')
report

,SalesRep,Month,Sales
0,BKK1101,JAN,97072
1,BKK1102,JAN,54712
2,BKK1103,JAN,48605
3,BKK2101,JAN,21028
4,BKK2102,JAN,46199
...,...,...,...
259,LTH101,DEC,0
260,LTH102,DEC,0
261,LTH103,DEC,0
262,LTH104,DEC,0


In [3]:
# sales target
from openpyxl import load_workbook 

targets = pd.DataFrame()
wb = load_workbook(filename='../datasets/Sales_targets.xlsx')
for ws in wb.sheetnames:
    print(ws)
    target = pd.read_excel('../datasets/Sales_targets.xlsx', header=4, sheet_name=ws, usecols=[i for i in range(0,13)])
    targets = pd.concat([targets, target])
    
targets.head()

Bangkok 1
Bangkok 2
UPPER TH
LOWER TH


,SalesRep,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,BKK1101,25000,27500.0,30250.0,33275.0,36602.5,40262.75,25000,27500.0,30250.0,33275.0,36602.5,40262.75
1,BKK1102,25000,27500.0,30250.0,33275.0,36602.5,40262.75,25000,27500.0,30250.0,33275.0,36602.5,40262.75
2,BKK1103,25000,27500.0,30250.0,33275.0,36602.5,40262.75,25000,27500.0,30250.0,33275.0,36602.5,40262.75
0,BKK2101,25000,27500.0,30250.0,33275.0,36602.5,40262.75,25000,27500.0,30250.0,33275.0,36602.5,40262.75
1,BKK2102,25000,27500.0,30250.0,33275.0,36602.5,40262.75,25000,27500.0,30250.0,33275.0,36602.5,40262.75


In [4]:
mlt_target = pd.melt(targets, id_vars='SalesRep', value_vars=targets.columns[1:],
                    var_name = 'Month', value_name = 'Target')
mlt_target

,SalesRep,Month,Target
0,BKK1101,JAN,25000.00
1,BKK1102,JAN,25000.00
2,BKK1103,JAN,25000.00
3,BKK2101,JAN,25000.00
4,BKK2102,JAN,25000.00
...,...,...,...
259,LTH101,DEC,11273.57
260,LTH102,DEC,11273.57
261,LTH103,DEC,11273.57
262,LTH104,DEC,11273.57


In [5]:
# target type casting
mlt_target['Target'] = mlt_target['Target'].round(0).astype(int)
mlt_target

,SalesRep,Month,Target
0,BKK1101,JAN,25000
1,BKK1102,JAN,25000
2,BKK1103,JAN,25000
3,BKK2101,JAN,25000
4,BKK2102,JAN,25000
...,...,...,...
259,LTH101,DEC,11274
260,LTH102,DEC,11274
261,LTH103,DEC,11274
262,LTH104,DEC,11274


#### Map sales target with report

In [6]:
compare = pd.merge(report, mlt_target, on=['SalesRep', 'Month'], how='inner')
compare.head(3)

,SalesRep,Month,Sales,Target
0,BKK1101,JAN,97072,25000
1,BKK1102,JAN,54712,25000
2,BKK1103,JAN,48605,25000


In [7]:
compare['Month'].unique()

array(['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC'], dtype=object)

In [8]:
# month order
mo_order = pd.DataFrame({'Month': ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP','OCT', 'NOV', 'DEC'],
                         'MonthNo': [i for i in range(1, 13)]})
mo_order

,Month,MonthNo
0,JAN,1
1,FEB,2
2,MAR,3
3,APR,4
4,MAY,5
5,JUN,6
6,JUL,7
7,AUG,8
8,SEP,9
9,OCT,10


In [9]:
compare = pd.merge(compare, mo_order, on=['Month'], how='inner')
compare.head(3)

,SalesRep,Month,Sales,Target,MonthNo
0,BKK1101,JAN,97072,25000,1
1,BKK1102,JAN,54712,25000,1
2,BKK1103,JAN,48605,25000,1


#### Calculate archivement ratio 

In [10]:
compare['AR'] = compare['Sales']/compare['Target']
compare.head(3)

,SalesRep,Month,Sales,Target,MonthNo,AR
0,BKK1101,JAN,97072,25000,1,3.88288
1,BKK1102,JAN,54712,25000,1,2.18848
2,BKK1103,JAN,48605,25000,1,1.94420


In [11]:
# pivot - long to wide format
pvt_compare = compare.pivot(index=['SalesRep'], columns=['MonthNo'], values=['AR']).reset_index(drop=False)
pvt_compare

SalesRep        AR                                                    \
MonthNo                  1         2         3         4         5         6   
0        BKK1101  3.882880  0.797818  2.209752  2.859174  0.606262  1.172441   
1        BKK1102  2.188480  1.446109  1.980959  1.586627  1.877906  0.476964   
2        BKK1103  1.944200  1.711345  0.871669  0.640661  1.563806  1.894742   
3        BKK2101  0.841120  3.019127  1.799636  1.680090  0.697730  2.195986   
4        BKK2102  1.847960  2.158509  0.832331  1.168475  1.816490  0.356854   
5        BKK2103  2.647560  2.132218  2.669917  2.174365  2.144360  0.281822   
6        BKK2104  3.385360  0.446000  2.624959  2.404237  0.790536  0.935772   
7        BKK2105  1.216440  2.771127  2.418380  0.535026  0.844576  0.539031   
8         LTH101  1.082714  0.217662  1.070248  0.848771  0.333789  0.211460   
9         LTH102  1.341429  1.003506  0.659504  0.933884  0.868085  0.559961   
10        LTH103  0.459571  0.729091  0.646989  0.500268  0.229388  0.532996   
11        LTH104  0.220714  0.507273  0.332349  0.353440  0.409601  0.535036   
12        LTH105  1.342000  0.560130  0.148288  0.725555  0.756757  0.390012   
13        UTH101  0.830143  1.272468  0.713695  0.843619  0.631086  0.241440   
14        UTH102  1.120571  0.771039  0.592798  0.773962  0.959606  0.623470   
15        UTH103  0.973000  0.258442  0.739433  0.891918  0.563177  0.622672   
16        UTH104  1.160571  0.783377  1.085714  0.399377  0.197092  0.158240   
17        UTH105  0.542857  0.637143  0.292680  0.823548  0.956776  0.669682   
18        UTH106  0.783857  0.221429  0.437898  0.516905  0.100107  0.223168   
19        UTH107  0.332571  0.134026  0.793625  0.958570  0.540150  0.454497   
20        UTH108  1.411571  0.978182  1.010744  0.423312  0.835886  0.349920   
21        UTH109  0.000000  0.000000  0.142385  0.496726  0.397697  0.300958   

                                            
MonthNo         7    8    9   10   11   12  
0        0.804040  0.0  0.0  0.0  0.0  0.0  
1        3.386840  0.0  0.0  0.0  0.0  0.0  
2        1.881160  0.0  0.0  0.0  0.0  0.0  
3        2.090520  0.0  0.0  0.0  0.0  0.0  
4        2.690080  0.0  0.0  0.0  0.0  0.0  
5        1.123920  0.0  0.0  0.0  0.0  0.0  
6        2.006000  0.0  0.0  0.0  0.0  0.0  
7        2.410400  0.0  0.0  0.0  0.0  0.0  
8        0.000000  0.0  0.0  0.0  0.0  0.0  
9        1.181429  0.0  0.0  0.0  0.0  0.0  
10       1.251857  0.0  0.0  0.0  0.0  0.0  
11       0.682429  0.0  0.0  0.0  0.0  0.0  
12       1.235571  0.0  0.0  0.0  0.0  0.0  
13       0.000000  0.0  0.0  0.0  0.0  0.0  
14       0.481714  0.0  0.0  0.0  0.0  0.0  
15       0.797429  0.0  0.0  0.0  0.0  0.0  
16       0.621429  0.0  0.0  0.0  0.0  0.0  
17       0.703571  0.0  0.0  0.0  0.0  0.0  
18       0.585714  0.0  0.0  0.0  0.0  0.0  
19       0.919000  0.0  0.0  0.0  0.0  0.0  
20       1.107143  0.0  0.0  0.0  0.0  0.0  
21       0.630429  0.0  0.0  0.0  0.0  0.0

In [12]:
data_bkk1 = pvt_compare[pvt_compare['SalesRep'].str.contains('BKK1')]
data_bkk1

SalesRep       AR                                                    \
MonthNo                 1         2         3         4         5         6   
0        BKK1101  3.88288  0.797818  2.209752  2.859174  0.606262  1.172441   
1        BKK1102  2.18848  1.446109  1.980959  1.586627  1.877906  0.476964   
2        BKK1103  1.94420  1.711345  0.871669  0.640661  1.563806  1.894742   

                                           
MonthNo        7    8    9   10   11   12  
0        0.80404  0.0  0.0  0.0  0.0  0.0  
1        3.38684  0.0  0.0  0.0  0.0  0.0  
2        1.88116  0.0  0.0  0.0  0.0  0.0

#### Generate excel report

In [13]:
# load modules
from openpyxl import Workbook
from openpyxl.styles import colors
from openpyxl.styles import Font, Color, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# create workbook
wb = Workbook()
# create worksheet
ws = wb.active

# create title and attibutes
ws['A1'] = 'Target Report'
ws['A1'].font = Font(bold=True)

ws['A3'] = 'Team'
ws['B3'] = 'BKK1'
ws['B3'].font = Font(color="00FF0000", italic=True)

ws['A5'] = 'SalesRep'
ws['A5'].fill = PatternFill("solid", fgColor="00FFFF00")
ws['A5'].font = Font(bold=True)

# create columns
for row in ws.iter_rows(min_row=5, max_row=5, min_col=2, max_col=13):
    i = 0
    for cell in row:
        cell.value = mo_order['Month'][i]
        cell.fill = PatternFill("solid", fgColor="00FFFF00")
        cell.font = Font(bold=True)
        i+=1

# load/add data
for r in dataframe_to_rows(data_bkk1, index=False, header=False):
    ws.append(r)

# number format - percent
for row in ws.iter_rows(min_row=6, max_row=8, min_col=2, max_col=13):
    for cell in row:
        cell.number_format = '0.00%'
    
# save to excel 
wb.save('../datasets/ar_report.xlsx')